<a href="https://colab.research.google.com/github/jmccrosky/regrets-reporter/blob/master/RegretsReporter_Final_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Install and import needed libraries

In [ ]:
!pip install -U google-cloud-bigquery google-cloud-bigquery-storage pyarrow dateparser gspread-dataframe gspread wbpy

In [ ]:
from google.colab import auth
from google.cloud import bigquery
from google.cloud import bigquery_storage
import google.auth

import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
from packaging import version
import itertools
import json
import matplotlib.pyplot as plt
import seaborn as sb

from datetime import datetime, timedelta
from plotly.offline import iplot
import plotly.graph_objs as go
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
import nltk
import dateparser
from google.colab import data_table

import gspread_dataframe as gd
import gspread
from oauth2client.client import GoogleCredentials
import statsmodels.formula.api as smf

import math

import statsmodels.formula.api as smf
import statsmodels.api as sm

from google.colab import data_table

import wbpy

## Set up data access

In [ ]:
# Access/Permissions bug: https://bugzilla.mozilla.org/show_bug.cgi?id=1654078
# Data collection bug: https://bugzilla.mozilla.org/show_bug.cgi?id=1644107
# Note that this code is published as documentation of our research, but data access is restricted to selected Mozilla employees and contractors.

auth.authenticate_user()
credentials, your_project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)
print('Authenticated')

project_id = "moz-fx-data-bq-regrets-report"
bq_client = bigquery.Client(project=project_id, credentials=credentials)
bq_storage_client = bigquery_storage.BigQueryReadClient(credentials=credentials)

gc = gspread.authorize(GoogleCredentials.get_application_default())

## Specify data end date

In [ ]:
end_date = '2021-5-31' # This cut-off date was used for final analysis performed in early June.

## Specify outliers for removal

In [ ]:
# Two volunteers issued an exceptional number of reports, suggesting they made use of the extension differently
# from the rest of the volunteer population.  They are excluded from analysis for better generalizability.

outliers = ["401b32e1-2ebd-4144-97c3-6d8e8e20bbc6", "c6c231d7-d3ae-4048-8feb-7710e4ba3305"]
outliers_bq_string = ", ".join(["'" + i + "'" for i in outliers])

## Utility Functions

In [ ]:
# Calculate confidence interval for proportion, either as limits or deltas from center

def my_prop_confint(count, nobs, for_plot=True):
  low, high = proportion_confint(count, nobs, method="agresti_coull")
  prop = count / nobs
  if for_plot:
    return (prop, prop - low, high - prop)
  return (prop, low, high)

## Load main data

In [ ]:
# Load data from BigQuery.  Note that data deletion requests are processed
# automatically, but we include a filter here on deletion requests just in case
# analysis is performed between when a request is issued and when it is processed.

main_query = f'''
WITH deletion_requests_t AS (
  SELECT
    event_metadata.client_timestamp AS timestamp,
    data_deletion_request.extension_installation_uuid AS installation_id,
  FROM
    `moz-fx-data-shared-prod.regrets_reporter.regrets_reporter_update`
  WHERE
    date(submission_timestamp) >= "2020-6-1" -- filter required by policy, but we want all deletion requests
    AND data_deletion_request IS NOT NULL
),
cleaned_t AS (
  SELECT *
  FROM
    `moz-fx-data-shared-prod.regrets_reporter.regrets_reporter_update` main_t
  LEFT JOIN
    deletion_requests_t
  ON
    main_t.event_metadata.extension_installation_uuid = deletion_requests_t.installation_id
    AND main_t.event_metadata.client_timestamp <= deletion_requests_t.timestamp
  WHERE
    deletion_requests_t.installation_id IS NULL
    AND date(submission_timestamp) <= "{end_date}"
)
SELECT
  CAST(submission_timestamp AS DATE) AS submission_date,
  event_metadata.extension_installation_uuid AS installation_id,
  CASE 
    WHEN regret_report IS NOT NULL THEN 'regret_report'
    WHEN youtube_usage_statistics_update IS NOT NULL THEN 'youtube_usage_statistics_update'
    ELSE '-' 
    END AS record_type,
  metadata.geo.country AS country,
  regret_report.report_data.parent_youtube_navigations_metadata AS trail_data,
  regret_report.report_data.report_data_uuid AS report_id,
  regret_report.form_step AS regret_form_step,
  regret_report.report_data.youtube_navigation_metadata.video_metadata.video_id AS regret_video_id,
  regret_report.report_data.youtube_navigation_metadata.video_metadata.video_title AS regret_video_title,
  regret_report.report_data.youtube_navigation_metadata.video_metadata.video_description AS regret_video_description,
  regret_report.report_data.youtube_navigation_metadata.video_metadata.view_count_at_navigation AS regret_video_view_count,
  regret_report.report_data.youtube_navigation_metadata.video_metadata.video_posting_date AS view_posting_date,
  regret_report.user_supplied_regret_categories AS regret_categories,
  regret_report IS NOT NULL AS regret_total,
  regret_report.report_data.youtube_navigation_metadata.via_search_results AS regret_search,
  regret_report.report_data.youtube_navigation_metadata.via_non_search_algorithmic_recommendations_content AS regret_recc,
  regret_report.report_data.youtube_navigation_metadata.via_recommendations_with_an_explicit_query_or_constraint_to_optimize_for AS regret_query,
  regret_report.report_data.parent_youtube_navigations_metadata AS regret_report_watch_history,
  youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.in_total AS watch_page_videos_total,
  youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.via_non_search_algorithmic_recommendations_content AS watch_page_videos_recc,
  youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.via_recommendations_with_an_explicit_query_or_constraint_to_optimize_for AS watch_page_videos_query,
  youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.via_search_results AS watch_page_videos_search,
  regret_report.report_data.youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.in_total AS report_watch_page_videos_total,
  regret_report.report_data.youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.via_non_search_algorithmic_recommendations_content AS report_watch_page_videos_recc,
  regret_report.report_data.youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.via_recommendations_with_an_explicit_query_or_constraint_to_optimize_for AS report_watch_page_videos_query,
  regret_report.report_data.youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.via_search_results AS report_watch_page_videos_search,
FROM
  cleaned_t
ORDER BY event_metadata.client_timestamp
'''

main_data = bq_client.query(main_query).result().to_dataframe(bqstorage_client=bq_storage_client)

In [ ]:
# Multiple records are recorded for each regret report, so filter to just the record with the most complete data.

reports_data = main_data.query("record_type=='regret_report'").sort_values("regret_form_step").drop_duplicates(subset="report_id", keep="last", ignore_index=True)

In [ ]:
# When there are duplicate reports, prefer retention of those that have categories reported.

reports_data["has_cat"] = [len(r.regret_categories) > 0 for i, r in reports_data.iterrows()]
duplicate_reports = reports_data[reports_data.sort_values(["has_cat"], ascending=False).duplicated(subset=["regret_video_id", "installation_id"], keep="last")].report_id

# Remove outliers

clean_reports_data = reports_data[~reports_data.installation_id.isin(outliers)]

# Some processing to attempt to determine how old the video was at reporting time.

clean_reports_data['parsed_date'] = [None if i is None else dateparser.parse(i.replace("Premiered", "").replace("on", "").replace("Streamed", "").replace("live", "")) for i in clean_reports_data.view_posting_date]
clean_reports_data['age'] = [None if r.parsed_date is None else (pd.to_datetime(r.submission_date) - pd.to_datetime(r.parsed_date).tz_localize(None)).days for i, r in clean_reports_data.iterrows()]

## Generate trail data

In [ ]:
# Data on number of views and age of video at reporting time for vidoes watched before regret reports.

all_trail_views = [
  None if j["video_metadata"] is None
  else j["video_metadata"]['view_count_at_navigation']
  for i, r in clean_reports_data.iterrows() for j in r.trail_data
]

all_trail_ages = [
  None if (None if (j["video_metadata"] is None or j["video_metadata"]['video_posting_date'] is None) else dateparser.parse(j["video_metadata"]['video_posting_date'].replace("Premiered", "").replace("on", "").replace("Streamed", "").replace("live", ""))) is None
  else (pd.to_datetime(r.submission_date) - pd.to_datetime(dateparser.parse(j["video_metadata"]['video_posting_date'].replace("Premiered", "").replace("on", "").replace("Streamed", "").replace("live", ""))).tz_localize(None)).days
  for i, r in clean_reports_data.iterrows() for j in r.trail_data
]

trails_data = pd.DataFrame({
    "age": all_trail_ages,
    "views": all_trail_views
})

## Generate user-level data

In [ ]:
# Generate data with user-level summaries of reports and YT usage.  Note that data
# deletion requests are processed
# automatically, but we include a filter here on deletion requests just in case
# analysis is performed between when a request is issued and when it is processed.
# Note also exclusion of data more than two months after a user's last report (or first appearance, if no reports were made)

uls2_query = f'''
WITH deletion_requests_t AS (
  SELECT
    event_metadata.client_timestamp AS timestamp,
    data_deletion_request.extension_installation_uuid AS installation_id,
  FROM
    `moz-fx-data-shared-prod.regrets_reporter.regrets_reporter_update`
  WHERE
    date(submission_timestamp) >= "2020-6-1" -- filter required by policy, but we want all deletion requests
    AND data_deletion_request IS NOT NULL
),
f_cleaned_t AS (
  SELECT *, event_metadata.extension_installation_uuid AS id
  FROM
    `moz-fx-data-shared-prod.regrets_reporter.regrets_reporter_update` main_t
  LEFT JOIN
    deletion_requests_t
  ON
    main_t.event_metadata.extension_installation_uuid = deletion_requests_t.installation_id
    AND main_t.event_metadata.client_timestamp <= deletion_requests_t.timestamp
  WHERE
    deletion_requests_t.installation_id IS NULL
    AND date(submission_timestamp) <= "{end_date}"
    AND event_metadata.extension_installation_uuid NOT IN ({outliers_bq_string})
),
last_report_t AS (
  SELECT
    id,
    MAX(CAST (submission_timestamp AS DATE)) AS last_report
  FROM
    f_cleaned_t
  WHERE
    regret_report IS NOT NULL
  GROUP BY
    id
),
first_seen_t AS (
  SELECT
    id,
    MIN(CAST (submission_timestamp AS DATE)) AS first_seen
  FROM
    f_cleaned_t
  GROUP BY
    id
),
date_filter_t AS (
  SELECT
    id,
    DATE_ADD(COALESCE(last_report, first_seen), INTERVAL 28 * 2 DAY) AS filter_date
  FROM
    first_seen_t LEFT JOIN last_report_t USING(id)
),
cleaned_t AS (
  SELECT *
  FROM f_cleaned_t JOIN date_filter_t USING(id)
  WHERE
    CAST(submission_timestamp AS DATE) <= filter_date
),
main_t AS (
SELECT
  metadata.geo.country AS country,
  CAST(submission_timestamp AS DATE) AS submission_date,
  id AS installation_id,
  regret_report.report_data.report_data_uuid AS report_id,
  regret_report.report_data.youtube_navigation_metadata.video_metadata.video_id AS regret_video_id,

  IF(regret_report IS NOT NULL, 1, 0) AS regret_total,
  regret_report.report_data.youtube_navigation_metadata.via_search_results AS regret_search,
  IF(regret_report.report_data.youtube_navigation_metadata.via_search_results = -1, 1, 0) AS regret_unknown,
  regret_report.report_data.youtube_navigation_metadata.via_non_search_algorithmic_recommendations_content AS regret_recc,
  regret_report.report_data.youtube_navigation_metadata.via_recommendations_with_an_explicit_query_or_constraint_to_optimize_for AS regret_query,

  youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.in_total AS watch_page_videos_total,
  youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.via_non_search_algorithmic_recommendations_content AS watch_page_videos_recc,
  youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.via_recommendations_with_an_explicit_query_or_constraint_to_optimize_for AS watch_page_videos_query,
  youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.via_search_results AS watch_page_videos_search,
  regret_report.report_data.youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.in_total AS report_watch_page_videos_total,
  regret_report.report_data.youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.via_non_search_algorithmic_recommendations_content AS report_watch_page_videos_recc,
  regret_report.report_data.youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.via_recommendations_with_an_explicit_query_or_constraint_to_optimize_for AS report_watch_page_videos_query,
  regret_report.report_data.youtube_usage_statistics_update.amount_of_youtube_videos_played_on_youtube_watch_pages.via_search_results AS report_watch_page_videos_search,
FROM
  cleaned_t

ORDER BY event_metadata.client_timestamp
),

id_t AS (
  SELECT
    installation_id
  FROM
    main_t
  GROUP BY
    installation_id
)

SELECT
  installation_id,

  ARRAY_AGG(country)[OFFSET(0)] AS country,
  COUNT(DISTINCT IF((regret_video_id IS NOT NULL) AND regret_total = 1, report_id, NULL)) AS regrets_total,
  GREATEST(COALESCE(MAX(watch_page_videos_total), 0), COALESCE(MAX(report_watch_page_videos_total), 0)) AS watch_page_videos_total,

  COUNT(DISTINCT IF((regret_video_id IS NOT NULL) AND regret_recc = 1, report_id, NULL)) AS regrets_recc,
  GREATEST(COALESCE(MAX(watch_page_videos_recc), 0), COALESCE(MAX(report_watch_page_videos_recc), 0)) AS watch_page_videos_recc,

  COUNT(DISTINCT IF((regret_video_id IS NOT NULL) AND regret_query = 1, report_id, NULL)) AS regrets_query,
  GREATEST(COALESCE(MAX(watch_page_videos_query), 0), COALESCE(MAX(report_watch_page_videos_query), 0)) AS watch_page_videos_query,

  COUNT(DISTINCT IF((regret_video_id IS NOT NULL) AND regret_search = 1, report_id, NULL)) AS regrets_search,
  GREATEST(COALESCE(MAX(watch_page_videos_search), 0), COALESCE(MAX(report_watch_page_videos_search), 0)) AS watch_page_videos_search,

  COUNT(DISTINCT IF((regret_video_id IS NOT NULL) AND regret_unknown = 1, report_id, NULL)) AS regrets_unknown,
  
FROM
  id_t
JOIN
  main_t
USING(installation_id)
GROUP BY
  installation_id
'''

uls2_data = bq_client.query(uls2_query).result().to_dataframe(bqstorage_client=bq_storage_client)

## Load API data

In [ ]:
# Load video data cached from YouTube API

api_query = '''
SELECT
  video_id,
  ANY_VALUE(language) AS language,
  ANY_VALUE(tags) AS tags,
  ANY_VALUE(comment_count) AS comment_count,
  ANY_VALUE(like_count) AS like_count,
  ANY_VALUE(dislike_count) AS dislike_count,
  ANY_VALUE(takedown) AS takedown,
  ANY_VALUE(blocked) AS blocked
FROM
  `moz-fx-data-shared-prod.regrets_reporter_analysis.yt_api_data_v4`
GROUP BY video_id
'''
api_data = bq_client.query(api_query).result().to_dataframe(bqstorage_client=bq_storage_client)
api_data['is_blocked'] = api_data.blocked.str.len() > 0

## Load takedown analysis data

In [ ]:
# Load data on takedown reasons for each taken down video (manually collected)

ws = gc.open("Takedown Videos").worksheet("Sheet1")
td_data = gd.get_as_dataframe(ws).dropna(axis=1, how='all').dropna(how='all')
td_data.loc[~td_data.subreason.isnull(), "reason"] = td_data.reason + ": " + td_data.subreason
td_data = td_data.drop("subreason", axis=1)
td_data.loc[td_data.reason.str.contains("copyright"), "reason"] = "Video unavailable: This video is no longer available due to a copyright claim"
td_data.loc[td_data.reason.str.contains("closed"), "reason"] = "Video unavailable: This video is no longer available because the YouTube account associated with this video has been terminated."
td_data.loc[td_data.reason.str.contains("scams"), "reason"] = "This video has been removed for violating YouTube's policy on spam, deceptive practices, and scams."
td_data.loc[td_data.reason.str[-1] != ".", "reason"] = td_data.reason + "."

In [ ]:
# Are any videos missing from the takedown analysis spreadsheet

print("\n".join(["https://www.youtube.com/watch?v=" + i for i in api_data[(api_data.takedown) & (~ api_data.video_id.isin(td_data.link.str[-11:]))].video_id]))

## Load country name data

In [ ]:
# Data with standardized mapping from country code to country name.

country_name_query = "select UPPER(raw_country) AS country_code, standardized_country AS country_name from `moz-fx-data-shared-prod.static.third_party_standardized_country_names`"
country_name_data = bq_client.query(country_name_query).result().to_dataframe(bqstorage_client=bq_storage_client)

## Load research assistant data

In [ ]:
# Data from reserch assistants who assessed regret reports.

ws = gc.open("RA Data").worksheet("Sheet1")
ra_data = gd.get_as_dataframe(ws).rename(columns={
    "PDF file number": "file_num",
    "Video ID": "regret_id",
    "URL": "video_url",
    "User ID": "ra_id",
    "Content": "content",
    "Trail": "trail",
    "Language": "language",
    "Should this video be on YouTube?": "on_yt",
    "If NO, under which Community Guideline Category?": "category",
    "Should this video be recommended?": "recommended",
    "If NO, under which Community Guideline Subcategory?": "subcategory",
    "What category does the trail fall into?": "trail_cat",
    "Any guess of why it was flagged as a regret?": "why_regret",
    "Do you think that this video talks about COVID-19? ": "pandemic",
    "Do you think that this video is particularly weird/bizarre/harmful?": "weird",
}).dropna(axis=1, how='all').dropna(how='all').fillna("unknown")

temp = ra_data.video_url
ra_data = ra_data.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
ra_data["video_url"] = temp

In [ ]:
ra_data.on_yt.replace([
    "--",
    "-",
    "---",
    "unavailable",
    "unspecified",
    "unsure",
    "unspecified - there is not even captions",
    "n/a (cannot watch it.)",
    "unable to view as video was made private"
], "unknown", inplace = True)

ra_data.on_yt.replace([
    "unspecified but most likely yes",
    "yed",
    "yes (with a change to the thumbnail, but video itself is fine)",
], "yes", inplace = True)

ra_data.on_yt.replace([
    "unspecified  - probably not",
], "no", inplace = True)

In [ ]:
ra_data.on_yt.value_counts()

In [ ]:
ra_data.recommended.replace([
    "--",
    "-",
    "---",
    "----",
    "unavailable",
    "unspecified",
    "unsure",
    "unspecified - there is not even captions",
    "unspecified - there are not even captions",
    "unsure as it is a news report",
], "unknown", inplace = True)

ra_data.recommended.replace([
    "unspecified but most likely yes",
    "yed",
    "yes (with a change to the thumbnail, but video itself is fine)",
], "yes", inplace = True)

ra_data.recommended.replace([
    "unspecified  - probably not",
], "no", inplace = True)

In [ ]:
ra_data.recommended.value_counts()

In [ ]:
ra_data.weird.replace([
    "--",
    "-",
    "---",
    "unsure",
    "unspecified",
    "unsure- if it is in fact distorting the truth, then yes it is harmful. but it is hard to tell.",
    "unavailable",
    "weird"
], "unknown", inplace = True)

ra_data[ra_data.weird.str.startswith("yes")]["weird"] = "yes"
ra_data[ra_data.weird.str.startswith("no")]["weird"] = "no"
ra_data.weird.replace([
    "possibly harmfu",
    "harmful",
    "bizarre.",
    "weird, but no breach of guidelines",
    "possibly harmful because of this lack of clarification about the paint",
    "possibly yes",
    "weird/ possibly harmful given the allegation.",
    "possibly harmful",
    "it can encourage harmful behavior",
    "unsure (video argues people shouldn't get tested because governments are using the information to make it's citizens docile- there is emerging evidence that this is in fact the case in some countries, so it is difficult to say if this is harnmful yet)",
    "to a moderate level, i won't say it should be recommended",
], "yes", inplace = True)

ra_data.weird.replace([
    "from my personal understanding it looks like an innocent conversation but i feel i am not qualified to judge.",
], "no", inplace = True)

In [ ]:
ra_data.weird.value_counts()

In [ ]:
ra_data.pandemic = ra_data.pandemic.str.lower()
ra_data.pandemic.replace([
    "--",
    "-",
    "---",
    "not sure",
    "unspecified",
    "unsure",
    "possible (live news broadcast)"
], "unknown", inplace = True)
ra_data.pandemic.replace([
    "sometimes",
    "probably yes",
    "yes.",
    "yes( from title)",
    "yes (talks about its effect on the economy)",
], "yes", inplace = True)
ra_data.pandemic.replace([
    "yes - the description says that the laws related to covid have been followed in the production of the video (worked out with google translate)",
    "unspecified, probably not",
    "unspecified - probably not",
    "no.",
    "probably not",
    "probably not - there were no captions",
    "briefly",
    "yes (mentions impact of covid-19 on tyson foods)",
    "briefly mentioned",
    "yes (briefly mentioned)",
    "no - title on pdf contains name of sitcom (my name is earl) from 2007",
    "no (but videos in trail do)",
    "yes (mentions it breifly when stating that she has not been travelling)",
    "yes; brief mentions in the discussion",
    "only briefly",
], "no", inplace = True)

In [ ]:
ra_data.pandemic.value_counts()

In [ ]:
ra_data['language'] = ra_data['language'].str.replace(r"\(.*\)","").str.strip()

ra_data.language.replace([
    "title in english, description in bosnian (google translate and video has no audio)",
], "bosnian", inplace = True)

ra_data.language.replace([
    "title in english but theres is no lyrics",
    "english in description",
    "egnlish",
    "englsih",
    "englsh",
], "english", inplace = True)

ra_data.language.replace([
    "majority of it is in hindi (google translate) and some is spoken in english.",
], "hindi", inplace = True)

ra_data.language.replace([
    "portugese",
], "portuguese", inplace = True)

ra_data.language.replace([
    "--",
    "-",
    "---",
    "telugu, hindi",
    "mostly in english and sometimes in hindi"
    "english and portuguese",
    "english and hindi",
    "multiple",
    "french and english",
    "several - ruassian, english and spanish among others",
    "english and portuguese",
    "video title is english but the video is spoken in hindi",
    "russian or english",
    "none or english",
    "description in english, i think audio is hungarian",
    "english title with song sung in romanian",
    "english and portugese",
    "unavailable",
    "english description. japanese song",
    "dutch & english",
    "mostly in english and sometimes in hindi",
    "english and malayalam",
    "tamil and english",
    "english & chinese",
    "video title is in english but video is spoken in romanian",
    "not sure. but the video was watched in indonesia.",
    "unsure",
    "english and maybe hindi",
    "no language",
    "dutch  & english",
    "no sound",
    "hindu  and english",
    "hindu and english",
    "english & japanese",
    "tamil  and english",
    "various, the creator is english",
    "english, japanese",
    "title in english, description in bosnian",
    "english and marathi"
], "unknown", inplace = True)

In [ ]:
data_table.DataTable(pd.DataFrame(ra_data.language.value_counts()))

In [ ]:
ra_data.category = ra_data.category.str.lower()

ra_data.category.replace([
    "nudity and sexual content policies",
    "unspecified - probably sensitive content",
    "nudity & sexual content policies",
], "sensitive content", inplace = True)

ra_data.category.replace([
    "spam, deceptive practices, and scams policies",
    "spam & deceptive practices",
    "spam and deceptive categories",
    "scams and deceptive practices",
    "spam or deceptive practices",
    "spam, deceptive practices and scams policies",
    "spam, deceptive practices, & scams policies",
], "spam and deceptive practices", inplace = True)

ra_data.category.replace([
    "others - provide false information ",
    "spread of misinformation",
    "other= misinformation= the conspiracy videos could misinform viewers in harmful ways.",
    "others - provide false information",
    "violent or dangerous content",
    "violent or graphic content policies",
    "violent and dangerous content",
    "violent or graphic content",
    "harmful and dangerous content, he recorded how he found the place for sleep, most of them were abandoned house but he have entered the construction site where work was conducting, it is really dangerous",
    "dangerous content",
    "hate speech",
    "covid-19 medical misinformation policy",
    "violent or dangerous content / sensitive content",
    "legal policies or harassment & cyberbullying",
    "violent ordangerous content",
    "harmful or dangerous content policies",
    "in my opinion it should fall under the dathegory of violent and dangerous content",
    "it does not sirectly fall under the cathegory dangerous contet, vbut i would say iut potentially puts people in dager if they find it something to try",
], "violent or dangerous content", inplace = True)

ra_data.category.replace([
    "others - provide false information ",
    "spread of misinformation",
    "violent or graphic content policies",
    "-",
    "--",
    "---",
    "unspecified",
    "other",
    "other: the voice over is creepy and awkward",
    "thumbnails policy",
    "no",
    "it doesn't fall under a particular category but could potentially impress people that don't know the outcome of the video. i would therefore not recommend it."
], "unknown", inplace = True)

ra_data.loc[ra_data.pandemic == "yes", "category"] = "pandemic-related"

In [ ]:
ra_data.category.value_counts()

In [ ]:
ra_data['trail_cat'] = ra_data['trail_cat'].str.replace(r"^(?!.*unrelated.*).*related.*$","related videos")
ra_data['trail_cat'] = ra_data['trail_cat'].str.replace(r"^.*unrelated.*$","unrelated videos")

ra_data.trail_cat.replace([
    "more videos like the reported video.",
    "more videos like the reported video",
    "vaguely related videos.",
    "video like the reported video.",
    "some videos like the reported video.",
    "more videos like the reported video and completely unrelated to the reported video",
    "other (some videos in the trail are similar but some are comletely different to the flagged one)",
    "more videos like reported videos",
    "similar video to the reported video (driving-related)",
    "more videos like reported video",
    "more videos like this one",
    "recommendation- more videos like the reported video",
    "- more videos like the reported video",
    "similar video to the reported video.",
    "more vidoes like the reported video",
    "more videos liek the reported video",
    "same channel",
    "more videos liek the one reported",
    "mostly similar videos",
    "more videos like reported vidoe",
    "more videos like the reported video (progressively more relevant content)",
    "recommendations from shocking and chiropractic videos so slightly similar.",
    "more vides like the reported video.",
    "more videos like the reported video, the reported video in the recommendations",
    "from the same channel",
    "recommendations- more videos like the reported video.",
    "more videos like the reported videos (i think)",
], "related videos", inplace = True)

ra_data.trail_cat.replace([
    "reported video is present in the trail (three times)",
    "reported video is present in the trail.",
    "video in trail",
    "reported video is present in the trail",
    "reported video present in the trail",
    "same video and more videos like reported video",
    "same video",
    "reported video is in the trail",
    "the reported video is in the recommendations",
    "reported video is present in the trail & more videos like the reported video",
    "reported video is present in the trail & other video is completely unreated to the reported video",
    "the reported video is featured in the trail.",
], "reported video is present in the trail", inplace = True)

ra_data.trail_cat.replace([
    "completely opposite political beliefs from the reported video.",
    "video like the reported video.",
    "some videos like the reported video.",
    "completely unrealted to the reported video",
], "unrelated videos", inplace = True)

ra_data.trail_cat.replace([
    "progressively more extreme content.",
    "progressivly more extreme content",
], "progressively more extreme content", inplace = True)

ra_data.trail_cat.replace([
    "other - unavailable video",
    "--",
    "-",
    "---",
    "direct  visit",
    "unclear",
    "direct",
    "other",
    "recommendation from unavailable video (youtube.com)",
    "recommendations",
    "unavailable",
    "accessed directly",
    "other - direct visit",
    "recommendation",
    "direct search",
    "search",
    "other - one unavailable video",
    "other (this was a comedy video)",
    "music videos",
    "other - videos unavailable",
    "searched content",
    "other: search results",
    "other (channel page)",
    "other (no trail)",
    "recommendation-",
    "direct visit",
    "other - unavailable videos",
    "other - unavailable videos",
    "videos in the trail are all unavailable.",
    "direct visit.",
    "no videos in the trail are still available.",
    "other channel page",
    "x2 unavailable videos",
    "direct visit - unavailable",
    "other: channel page",
], "unknown", inplace = True)

In [ ]:
ra_data.trail_cat.value_counts()

In [ ]:
data_table.DataTable(pd.DataFrame(ra_data.subcategory.value_counts()))

In [ ]:
ra_data['video_id'] = ra_data.video_url.str[-11:]

In [ ]:
ra_joined = pd.merge(left=clean_reports_data, right=ra_data, how='left', left_on='report_id', right_on='regret_id')

# Results

## Total users and reports

In [ ]:
total_users = len(main_data.installation_id.unique())
total_reports = len(reports_data)
total_reports_dedup = len(reports_data[~ reports_data.report_id.isin(duplicate_reports)])
distinct_videos = len(clean_reports_data.regret_video_id.unique())
total_analysis_reports = len(clean_reports_data)
total_analysis_reports_dedup = len(clean_reports_data[~ clean_reports_data.report_id.isin(duplicate_reports)])
total_users_with_report = len(reports_data.installation_id.unique())
total_countries = len(list(filter(None, main_data.country.unique())))
total_countries_with_report = len(list(filter(None, reports_data.country.unique())))
first_data_date = main_data.submission_date.min()
ra_data_count = len(ra_data.query("on_yt!='unknown'"))

print(f"Total users: {total_users}; Total reports: {total_reports}; Total reports removing outliers: {total_analysis_reports}")
print(f"Total users with report: {total_users_with_report}; Total countries: {total_countries}; Total countries with report: {total_countries_with_report}")
print(f"First data date: {first_data_date}")
print(f"Total Reports without dupes: {total_reports_dedup}; Total Reports without dupes or outliers: {total_analysis_reports_dedup}")
print(f"RA videos: {ra_data_count}; distinct videos: {distinct_videos}; RA rate: {ra_data_count / distinct_videos}")

In [ ]:
# Outlier reports

reports_data[reports_data.installation_id.isin(outliers)].installation_id.value_counts()

In [ ]:
# Reports distribution

data_table.DataTable(pd.DataFrame(reports_data.installation_id.value_counts()))

## Engagement of regret videos

In [ ]:
median_report_views = np.median([i for i in clean_reports_data.regret_video_view_count if (~np.isnan(i) and i > 0)])
median_trail_views = np.median([i for i in all_trail_views if ( i is not None and ~np.isnan(i) and i > 0)])
median_report_age = np.median([i for i in clean_reports_data.age if (~np.isnan(i) and i >= 0)])
median_trail_age = np.median([i for i in all_trail_ages if i is not None])
temp_valid_report_videos = clean_reports_data.query("regret_video_view_count > 0 & age > 0")
median_report_views_per_day = np.median([i.regret_video_view_count / i.age for x, i in temp_valid_report_videos.iterrows()])
temp_valid_trail_videos = trails_data.query("views > 0 & age > 0")
median_trail_views_per_day = np.median([i.views / i.age for x, i in temp_valid_trail_videos.iterrows()])

print(f"Trail engagement: {median_trail_views_per_day}; Report engagement: {median_report_views_per_day}; Factor: {median_report_views_per_day / median_trail_views_per_day}")

## Regret rate by entry point

In [ ]:
rate_recc = my_prop_confint(uls2_data.regrets_recc.sum(), uls2_data.watch_page_videos_recc.sum(), for_plot=False)
rate_search = my_prop_confint(uls2_data.regrets_search.sum(), uls2_data.watch_page_videos_search.sum(), for_plot=False)

print(f"Recc Rate: {rate_recc[0] * 10000}; Search Rate: {rate_search[0] * 10000}; Factor: {rate_recc[0] / rate_search[0]}")

## Takedown analysis

In [ ]:
# Assessment of whether each YouTube-provided reason for takedown indicates violative content.

reason_map = {
  'Available now.': 'no',
  'Invalid.': 'no',
  "This video has been removed for violating YouTube's Community Guidelines.": 'yes',
  "This video has been removed for violating YouTube's Terms of Service.": 'yes',
  "This video has been removed for violating YouTube's policy on harassment and bullying.": 'yes',
  "This video has been removed for violating YouTube's policy on hate speech. Learn more about combating hate speech in your country.": 'yes',
  "This video has been removed for violating YouTube's policy on spam, deceptive practices, and scams.": 'yes',
  "This video has been removed for violating YouTube's policy on violent or graphic content.": 'yes',
  'Video unavailable.': 'maybe',
  'Video unavailable: This video has been removed by the uploader.': 'no',
  'Video unavailable: This video is no longer available because the YouTube account associated with this video has been terminated.': 'yes',
  'Video unavailable: This video is no longer available due to a copyright claim.': 'no',
  'Video unavailable: This video is private.': 'no'
}

In [ ]:
temp_takedown_data = td_data
temp_takedown_data['violative'] = [reason_map[r] for r in temp_takedown_data.reason]
temp_takedown_data['recc'] = [len(clean_reports_data[(clean_reports_data.regret_recc == 1) & (clean_reports_data.regret_video_id == r[-11:])]) > 0 for r in temp_takedown_data.link]
recc_takedowns = len(temp_takedown_data.query("recc"))
total_recc = len(clean_reports_data.query("regret_recc == 1").regret_video_id.unique())

print(f"Recc takedowns: {recc_takedowns}; Recc takedown rate: {recc_takedowns / total_recc}")

In [ ]:
# Takedown reasons among taken down regrets that were recommended to our users.

temp_recc_takedown_data = temp_takedown_data.query("recc")
temp_recc_takedown_data.reason.value_counts()

In [ ]:
temp_recc_takedown_data.query("recc").reason.value_counts(normalize=True)

In [ ]:
labels = temp_recc_takedown_data.violative.value_counts().index.to_list()
values = [temp_recc_takedown_data.violative.value_counts()[k] for k in temp_recc_takedown_data.violative.value_counts().index.to_list()]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+value+percent')])

fig.update_layout(
    title="Takedown violativeness",
    width=650,
)

ws = gc.open("Graph Data").worksheet("takedown violativeness")
gd.set_with_dataframe(ws, pd.DataFrame({
    "label": labels,
    "value": values,
}))

fig.show()

In [ ]:
temp_recc_viol_takedown_data = temp_takedown_data.query("recc & violative != 'no'")
recc_violative_takedowns = len(temp_recc_viol_takedown_data)
total_recc = len(clean_reports_data.query("regret_recc == 1").regret_video_id.unique())

print(f"Recc violative takedowns: {recc_violative_takedowns}; Recc violative takedown rate: {recc_violative_takedowns / total_recc}")

In [ ]:
viol_recc_takedown_rate = len(temp_recc_viol_takedown_data) / len(clean_reports_data.query("regret_recc==1").regret_video_id.unique())
viol_recc_takedown_views = clean_reports_data[clean_reports_data.regret_video_id.isin(temp_recc_viol_takedown_data.link.str[-11:])].regret_video_view_count.sum()
viol_recc_takedown_count = clean_reports_data[clean_reports_data.regret_video_id.isin(temp_recc_viol_takedown_data.link.str[-11:])].regret_video_view_count.count()
viol_recc_takedown_days = clean_reports_data[clean_reports_data.regret_video_id.isin(temp_recc_viol_takedown_data.link.str[-11:])].age.sum()

print(f"Recc violative takedown rate: {viol_recc_takedown_rate}; Recc violative takedown views: {viol_recc_takedown_views}")
print(f"Recc violative takedown views per: {viol_recc_takedown_views / viol_recc_takedown_count}; Recc violative takedown days per: {viol_recc_takedown_days / viol_recc_takedown_count}")

## Rates by country

In [ ]:
# Classifying primary language or each country as English or non-English from CIA world factbook
# See report for more details.

langmap = {
    "AR": 0,
    "AU": 1,
    "BE": 0,
    "BR": 0,
    "CA": 1,
    "DE": 0,
    "ES": 0,
    "FI": 0,
    "FR": 0,
    "GB": 1,
    "GR": 0,
    "HU": 0,
    "IN": 0,
    "IT": 0,
    "JP": 0,
    "MX": 0,
    "NL": 0,
    "PH": 0,
    "PL": 0,
    "RO": 0,
    "RU": 0,
    "SE": 0,
    "TR": 0,
    "UA": 0,
    "US": 1,
    "IE": 1,
    "ZA": 0,
    "NZ": 1,
    "ID": 0,
    "BD": 0,
    "SG": 1,
    "PT": 0,
    "MY": 0,
    "BG": 0,
    "CL": 0,
    "NP": 0,
    "RS": 0,
    "CO": 0,
    "TW": 0,
    "CH": 0,
    "SI": 0,
    "DK": 0,
    "AT": 0,
    "EE": 0,
    "CZ": 0,
    "VN": 0,
    "NO": 0,

}

# Countries that YT reports focusing on (see report)
focus = ["US", "GB", "IR", "ZA", "BR", "FR", "DE", "MX", "ES"]
focus_en = ["US", "GB", "IR"]

In [ ]:
# Set up country data

countries = list(filter(None, uls2_data.country.unique()))

def get_name(country_code):
  if country_code == "US":
    return "United States"  # Needed as this table uses USA but we need to link to data that uses United States
  try:
    return country_name_data.query("country_code==@country_code").country_name.item()
  except:
    return "unknown"

def get_population(country_code):
  try:
    return wbpy.IndicatorAPI().get_dataset("SP.POP.TOTL", [country_code], date="2019:2020").as_dict()[country_code]["2019"]
  except:
    return None

am_data = pd.read_csv(
  f"https://drive.google.com/uc?export=download&id=1rb-ioB9OZtL_aHj96xHPh55k-etxVUA5", # 2019 from statista
  sep=';',
  thousands=','
)

country_data = pd.DataFrame({
    "country_code" : countries,
    "country_name": [get_name(c) for c in countries],
    "regret_rate_num": [uls2_data.query("country == @c")['regrets_total'].sum() for c in countries],
    "pandemic_rate_num": [len(ra_joined.query("pandemic == 'yes' & (on_yt=='no' | recommended=='no') & country==@c")) for c in countries],
    "regret_rate_den": [uls2_data.query("country == @c")["watch_page_videos_total"].sum() for c in countries],
    "language": [langmap[c] if c in langmap else -1 for c in countries],
    
    "population": [get_population(c) for c in countries],
    "ad_market": [
      am_data[am_data.Country == get_name(c)].Amount.astype(float).item()
      if len(am_data[am_data.Country == get_name(c)].Amount) == 1
      else None
      for c in countries]
})

country_data.loc[:, 'regret_rate'] = country_data.regret_rate_num / country_data.regret_rate_den
country_data.loc[:, 'ci'] = [my_prop_confint(i.regret_rate_num, i.regret_rate_den) if i.regret_rate_den > 0 else None for x, i in country_data.iterrows()]
country_data.loc[:, 'variance'] = [None if i.ci is None else i.ci[1] + i.ci[2] for r, i in country_data.iterrows()]
country_data.loc[:, 'ci'] = [my_prop_confint(i.regret_rate_num, i.regret_rate_den) if i.regret_rate_den > 0 else None for x, i in country_data.iterrows()]
country_data.loc[:, 'ad_market_per_capita'] = country_data.ad_market / country_data.population
country_data.loc[:, 'log_ad_market_per_capita'] = np.log(country_data.ad_market_per_capita)
country_data.loc[:, 'log_ad_market'] = np.log(country_data.ad_market)
country_data = country_data.sort_values("regret_rate")

In [ ]:
temp = country_data.query("variance < 0.0011")[["country_name", "regret_rate", "ci", "language", ]].dropna()
colors = {0: "Green", 1: "Blue", -1: "Red"}
fig = go.Figure(data=[
  go.Bar(
        y=[temp.iloc[i].country_name],
        x=[temp.iloc[i].regret_rate * 10000],
        error_x = dict(
            type='data',
            symmetric=False,
            array=[temp.iloc[i].ci[2] * 10000],
            arrayminus = [temp.iloc[i].ci[1] * 10000]
        ),
        orientation='h',
        marker=dict(color=colors[temp.iloc[i].language]),
        name='Primary-non-English' if temp.iloc[i].language==0 else 'Primary-English' if temp.iloc[i].language==1 else 'unknown',
        legendgroup='Primary-non-English' if temp.iloc[i].language==0 else 'Primary-English' if temp.iloc[i].language==1 else 'unknown',
        showlegend = temp.iloc[i].country_name in ["Brazil", "United States"]
    )
  for i in range(len(temp))
])

fig.update_layout(
    title="Regret Rates by Country",
    xaxis = dict(title = "Regrets per 10000 videos"),
    width = 800,
    height = 900                
)

ws = gc.open("Graph Data").worksheet("country rates")
gd.set_with_dataframe(ws, pd.DataFrame({
    "name": temp.country_name,
    "rate": [i * 10000 for i in temp.regret_rate],
    "error_bar_delta_plus": [i[2] * 10000 for i in temp.ci],
    "error_bar_delta_minus": [i[1] * 10000 for i in temp.ci],
    "language_1_is_eng_0_is_other": temp.language,
}))

fig.show()

In [ ]:
lang_classified_report_proportion = country_data.query("language != -1").regret_rate_num.sum() / country_data.regret_rate_num.sum()

temp = country_data.query("language==1")
english_regret_rate = my_prop_confint(temp.regret_rate_num.sum(), temp.regret_rate_den.sum(), for_plot=False)

temp = country_data.query("language==0")
non_english_regret_rate = my_prop_confint(temp.regret_rate_num.sum(), temp.regret_rate_den.sum(), for_plot=False)

print(f"Proportion of reports with classified language: {lang_classified_report_proportion}")
print(f"English rate: {english_regret_rate}; nonEnglish rate: {non_english_regret_rate}")

In [ ]:
lang_classified_report_proportion = country_data.query("language != -1").pandemic_rate_num.sum() / country_data.pandemic_rate_num.sum()

temp = country_data.query("language==1")
english_regret_rate = my_prop_confint(temp.pandemic_rate_num.sum(), temp.regret_rate_den.sum(), for_plot=False)

temp = country_data.query("language==0")
non_english_regret_rate = my_prop_confint(temp.pandemic_rate_num.sum(), temp.regret_rate_den.sum(), for_plot=False)

print(f"PANDEMIC Proportion of reports with classified language: {lang_classified_report_proportion}")
print(f"PANDEMIC English rate: {english_regret_rate}; nonEnglish rate: {non_english_regret_rate}")

In [ ]:
tempa = country_data[country_data.country_code.isin(focus_en)]
focus_regret_rate = my_prop_confint(tempa.regret_rate_num.sum(), tempa.regret_rate_den.sum(), for_plot=False)

tempb = country_data[~country_data.country_code.isin(focus_en)]
non_focus_regret_rate = my_prop_confint(tempb.regret_rate_num.sum(), tempb.regret_rate_den.sum(), for_plot=False)

print(f"Focus rate: {focus_regret_rate}; non-focus rate: {non_focus_regret_rate}")
print(proportions_ztest([tempa.regret_rate_num.sum(), tempb.regret_rate_num.sum()], [tempa.regret_rate_den.sum(), tempb.regret_rate_den.sum()]))

## Deactivation criteria


In [ ]:
%%bigquery long_data --project moz-fx-data-bq-regrets-report
WITH deletion_requests_t AS (
  SELECT
    event_metadata.client_timestamp AS timestamp,
    data_deletion_request.extension_installation_uuid AS installation_id,
  FROM
    `moz-fx-data-shared-prod.regrets_reporter.regrets_reporter_update`
  WHERE
    date(submission_timestamp) >= "2020-06-01"  -- Filter on submission_timestamp is required by policy.
    AND data_deletion_request IS NOT NULL
),
cleaned_t AS (
  SELECT *
  FROM
    `moz-fx-data-shared-prod.regrets_reporter.regrets_reporter_update` main_t
  LEFT JOIN
    deletion_requests_t
  ON
    main_t.event_metadata.extension_installation_uuid = deletion_requests_t.installation_id
    AND main_t.event_metadata.client_timestamp <= deletion_requests_t.timestamp
  WHERE
    deletion_requests_t.installation_id IS NULL
    AND date(submission_timestamp) <= "2021-5-31"
),

active_days_t AS (
  SELECT
    event_metadata.extension_installation_uuid AS id,
    ARRAY_AGG(DISTINCT CAST(submission_timestamp AS DATE) ORDER BY CAST(submission_timestamp AS DATE)) AS active_days
  FROM
    cleaned_t
  GROUP BY
    id
),
report_days_t AS (
  SELECT
    event_metadata.extension_installation_uuid AS id,
    ARRAY_AGG(DISTINCT CAST(submission_timestamp AS DATE) ORDER BY CAST(submission_timestamp AS DATE)) AS report_days
  FROM
    cleaned_t
  WHERE
    regret_report IS NOT NULL
  GROUP BY
    id
)
SELECT * FROM active_days_t LEFT JOIN report_days_t USING(id)

In [ ]:
long_data["last_report"] = [max(days, default=None) for days in long_data.report_days]
long_data["last_active"] = [max(days) for days in long_data.active_days]
long_data["first_report"] = [min(days, default=None) for days in long_data.report_days]
long_data["first_active"] = [min(days) for days in long_data.active_days]

In [ ]:
temp = long_data[~long_data.first_report.isnull()]
for i in range(5):
  print(len(temp[temp.first_report <= temp.first_active + pd.Timedelta(days=28*i)]) / len(temp))

## Regret Categories

In [ ]:
evaluated_videos = len(ra_joined[ra_joined.on_yt.isin(["yes", "no"]) & ra_joined.recommended.isin(["yes", "no"])])
noton_videos = len(ra_joined[ra_joined.on_yt.isin(["no"])])
notrecc_videos = len(ra_joined[ra_joined.recommended.isin(["no"])])
neither_videos = evaluated_videos = len(ra_joined[ra_joined.on_yt.isin(["no"]) & ra_joined.recommended.isin(["no"])])

print(f"Total evaluated reports: {evaluated_videos}; Videos that should not be on YT: {noton_videos}; Videos that should not be recommended: {notrecc_videos}; Neither videos: {neither_videos}")

## Pandemic languages

In [ ]:
english_rate_for_covid = len(ra_data.query("language=='english' & pandemic=='yes' & (on_yt=='no' | recommended=='no')")) / len(ra_data.query("pandemic=='yes' & language != 'unknown' & (on_yt=='no' | recommended=='no')"))
english_rate_for_other = len(ra_data.query("language=='english' & pandemic=='no' & (on_yt=='no' | recommended=='no')")) / len(ra_data.query("pandemic=='no' & language != 'unknown' & (on_yt=='no' | recommended=='no')"))

pandemic_rate_for_english = len(ra_data.query("language=='english' & pandemic=='yes' & (on_yt=='no' | recommended=='no')")) / len(ra_data.query("language=='english' & (on_yt=='no' | recommended=='no')"))
pandemic_rate_for_other = len(ra_data.query("language!='english' & language != 'unknown' & pandemic=='yes' & (on_yt=='no' | recommended=='no')")) / len(ra_data.query("language!='english' & language != 'unknown' & (on_yt=='no' | recommended=='no')"))

In [ ]:
print(f"English rate for pandemic-related regrets: {english_rate_for_covid}; English rate for other regrets: {english_rate_for_other}")
print(f"Pandemic rate for English: {pandemic_rate_for_english}; Pandemic rate for non-English: {pandemic_rate_for_other}")

In [ ]:
print(proportions_ztest([len(ra_data.query("language=='english' & pandemic=='yes' & (on_yt=='no' | recommended=='no')")), len(ra_data.query("language!='english' & language!='unknown' & pandemic=='yes' & (on_yt=='no' | recommended=='no')"))],
                        [len(ra_data.query("pandemic!='unknown' & language=='english' & (on_yt=='no' | recommended=='no')")), len(ra_data.query("pandemic!='unknown' & language!='english' & language!='unknown' & (on_yt=='no' | recommended=='no')"))]))

In [ ]:
print(proportions_ztest([len(ra_data.query("language=='english' & pandemic=='yes' & (on_yt=='no' | recommended=='no')")), len(ra_data.query("language=='english' & pandemic=='no' & (on_yt=='no' | recommended=='no')"))],
                        [len(ra_data.query("pandemic=='yes' & language != 'unknown' & (on_yt=='no' | recommended=='no')")), len(ra_data.query("pandemic=='no' & language != 'unknown' & (on_yt=='no' | recommended=='no')"))]))

## Trail categories

In [ ]:
ra_data.trail_cat.value_counts()

In [ ]:
labels = [
  "related videos",
  "unrelated videos",
  "reported video is present in the trail",
  "progressively more extreme content",
]
values = [
    ra_data.trail_cat.value_counts()["related videos"],
    ra_data.trail_cat.value_counts()["unrelated videos"],
    ra_data.trail_cat.value_counts()["reported video is present in the trail"],
    ra_data.trail_cat.value_counts()["progressively more extreme content"],

]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent')])

fig.update_layout(
    title="Trail Categories",
    width=1250,
    height = 800
)

fig.show()

## Regret source

In [ ]:
recc_count = len(clean_reports_data.query("regret_recc==1"))
search_count = len(clean_reports_data.query("regret_search==1"))
other_count = len(clean_reports_data) - recc_count - search_count

labels = ['Recommendation','Search','Other']
values = [recc_count, search_count, other_count]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent')])

fig.update_layout(
    title="Regret Sources",
    width=600
)

ws = gc.open("Graph Data").worksheet("regret source")
gd.set_with_dataframe(ws, pd.DataFrame({
    "label": labels,
    "value": values,
}))

fig.show()

## Extreme examples and category examples

In [ ]:
ra_data.category.value_counts()

In [ ]:
for c in ["violent or dangerous content", "spam and deceptive practices", "sensitive content", "regulated goods", "pandemic-related"]:
    ws = gc.open("Weird/Harmful Examples").worksheet(c)
    gd.set_with_dataframe(ws, ra_data.query(f"weird=='yes' & category=='{c}'")[["regret_id", "content", "subcategory", "why_regret", "video_url"]])